<a href="https://colab.research.google.com/github/gunjak/Placement-Assignment_Gunja/blob/main/CV_que1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files

In [3]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gunjakumari","key":"fc66adebe0a688adcd9811cbfee78811"}'}

In [4]:
!mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d 'misrakahmed/vegetable-image-dataset'

 98% 521M/534M [00:06<00:00, 86.2MB/s]
100% 534M/534M [00:06<00:00, 90.9MB/s]


In [ ]:
! mkdir train
! unzip /content/vegetable-image-dataset.zip -d train

In [9]:
import cv2
image=cv2.imread('/content/train/Vegetable Images/train/Bean/0026.jpg')
image.shape

(224, 224, 3)

In [10]:
import os
import torch
import torchvision
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
from torch import nn 
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
import random
torch.manual_seed(0)
from tqdm.auto import tqdm

In [11]:
train_dir='/content/train/Vegetable Images/train'
test_dir='/content/train/Vegetable Images/test'

In [12]:
import glob
classes=['Bean','Bitter_Gourd','Bottle_Gourd','Brinjal','Broccoli','Cabbage',
'Capsicum','Carrot','Cauliflower','Cucumber','Papaya','Potato','Pumpkin',"Radish",'Tomato']
count=0
train_files=[]
while count!=15:
  for name in glob.glob(f'{train_dir}/{classes[count]}/*'):
    train_files.append(name)
  count+=1
  
test_files=[]
count=0
while count!=15:
  for name in glob.glob(f'{test_dir}/{classes[count]}/*'):
    test_files.append(name)
  count+=1

In [13]:
train_files[0]

'/content/train/Vegetable Images/train/Bean/1147.jpg'

In [14]:
random.shuffle(train_files)

In [15]:
os.path.normpath('/content/train/Vegetable Images/train/Bean/0026.jpg').split(os.sep)[-2]

'Bean'

In [16]:
class VegetablesDataset(Dataset):
    def __init__(self, images_filepaths, transform=None):
        self.images_filepaths = images_filepaths
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if os.path.normpath(image_filepath).split(os.sep)[-2] == classes[0]:
            label = 0.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[1]:
            label = 1.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[2]:
            label = 2.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[3]:
            label = 3.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[4]:
            label = 4.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[5]:
            label = 5.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[6]:
            label = 6.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[7]:
            label = 7.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[8]:
            label = 8.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[9]:
            label = 9.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[10]:
            label = 10.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[11]:
            label = 11.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[12]:
            label = 12.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[13]:
            label = 13.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[14]:
            label = 14.0
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        return image, label
        
correct_images_filepaths = [i for i in train_files if cv2.imread(i) is not None]


In [17]:
train_transform = A.Compose(
    [
        A.SmallestMaxSize(max_size=160),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomCrop(height=128, width=128),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)
test_transform = A.Compose(
    [
        A.RandomCrop(height=128, width=128),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)


In [18]:
train_dataset = VegetablesDataset(images_filepaths=train_files, transform=train_transform)
test_dataset=VegetablesDataset(images_filepaths=test_files, transform=test_transform)

In [19]:
val_size = 5000
train_size = len(train_dataset) - val_size

train_ds, val_ds = random_split(train_dataset, [train_size, val_size])
len(train_ds), len(val_ds)

(10000, 5000)

In [20]:
params = {
    "device": "cuda",
    "num_workers": 4}
NUM_EPOCHS = 20
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

In [21]:
train_dl = DataLoader(
    train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=params["num_workers"], pin_memory=True,
)
val_dl=DataLoader(val_ds,batch_size=BATCH_SIZE, shuffle=True, num_workers=params["num_workers"], pin_memory=True,)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [22]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
device=get_default_device()

In [23]:

class TinyVGG1(nn.Module):
    def __init__(self, input_shape: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=32, 
                    kernel_size=3, 
                    stride=1, 
                    padding=0),  
          nn.ReLU(),
          nn.Conv2d(in_channels=32, 
                    out_channels=16,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
        )
        self.conv_block_2 = nn.Sequential(
          nn.Conv2d(16, 16, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(16, 10, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)

        )
        self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.LazyLinear(
                    out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x

In [24]:
model=TinyVGG1(
    input_shape=3,
    output_shape=len(classes)
).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [25]:
NUM_GPU = torch.cuda.device_count()
if NUM_GPU > 1:
	model = nn.DataParallel(model)
    
# initialize loss function, optimizer, and gradient scaler

In [26]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [27]:
results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }
for epoch in tqdm(range(10)):  
  model.train()
  train_loss, train_acc = 0, 0
  for batch, (X, y) in enumerate(train_dl):
          # Send data to target device
    y=y.type(torch.LongTensor)
    X, y = X.to(device), y.to(device)

          # 1. Forward pass
    y_pred = model(X)
    
          # 2. Calculate  and accumulate loss
    loss = loss_fn(y_pred, y)
    writer.add_scalar("Loss/train", loss, epoch)
    train_loss += loss.item() 

          # 3. Optimizer zero grad
    optimizer.zero_grad()

          # 4. Loss backward
    loss.backward()

          # 5. Optimizer step
    optimizer.step()

          # Calculate and accumulate accuracy metric across all batches
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)

      # Adjust metrics to get average loss and accuracy per batch 
  train_loss = train_loss / len(train_dl)
  train_acc = train_acc / len(train_dl)
  model.eval() 

    # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

    # Turn on inference context manager
  with torch.inference_mode():
        # Loop through DataLoader batches
    for batch, (X, y) in enumerate(val_dl):
      y=y.type(torch.LongTensor)
            # Send data to target device
      X, y = X.to(device), y.to(device)

            # 1. Forward pass
      test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

            # Calculate and accumulate accuracy
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch 
  test_loss = test_loss / len(val_dl)
  test_acc = test_acc / len(val_dl)
  print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
  results["train_loss"].append(train_loss)
  results["train_acc"].append(train_acc)
  results["test_loss"].append(test_loss)
  results["test_acc"].append(test_acc)
  
writer.flush()

    

  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 1 | train_loss: 1.7489 | train_acc: 0.4185 | test_loss: 1.2785 | test_acc: 0.5818
Epoch: 2 | train_loss: 1.1345 | train_acc: 0.6308 | test_loss: 1.0456 | test_acc: 0.6559
Epoch: 3 | train_loss: 0.9267 | train_acc: 0.7005 | test_loss: 0.8813 | test_acc: 0.7140
Epoch: 4 | train_loss: 0.7965 | train_acc: 0.7411 | test_loss: 0.7863 | test_acc: 0.7305
Epoch: 5 | train_loss: 0.7339 | train_acc: 0.7571 | test_loss: 0.7381 | test_acc: 0.7532
Epoch: 6 | train_loss: 0.6753 | train_acc: 0.7735 | test_loss: 0.6984 | test_acc: 0.7773
Epoch: 7 | train_loss: 0.6124 | train_acc: 0.7981 | test_loss: 0.6429 | test_acc: 0.7874
Epoch: 8 | train_loss: 0.5774 | train_acc: 0.8179 | test_loss: 0.5685 | test_acc: 0.8189
Epoch: 9 | train_loss: 0.5532 | train_acc: 0.8189 | test_loss: 0.5638 | test_acc: 0.8213
Epoch: 10 | train_loss: 0.5267 | train_acc: 0.8284 | test_loss: 0.5536 | test_acc: 0.8205


In [28]:
torch.save(model.state_dict(),'model_script.pt')
model = TinyVGG1(3,15)
model.load_state_dict(torch.load('model_script.pt'))
model.to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


TinyVGG1(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): LazyLinear(in_features=0, out_features=15, bias=True)
  )
)

In [29]:
test_images_filepaths=['/content/train/Vegetable Images/test/Broccoli/1001.jpg','/content/train/Vegetable Images/test/Cabbage/1201.jpg']
class VegetablesInfDataset(Dataset):
    def __init__(self, images_filepaths, transform=None):
        self.images_filepaths = images_filepaths
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        return image
t_dataset=VegetablesInfDataset(images_filepaths=test_images_filepaths,transform=test_transform)
t_loader = DataLoader(
    t_dataset, batch_size=1 ,shuffle=False, num_workers=params["num_workers"], pin_memory=True,
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [30]:
dic=['Bean','Bitter_Gourd','Bottle_Gourd','Brinjal','Broccoli','Cabbage','Capsicum','Carrot','Cauliflower','Cucumber','Papaya','Potato','Pumpkin','Radish','Tomato']
model = model.eval()
predicted_labels = []
with torch.no_grad():
    for batch, images in enumerate(t_loader):
        images = images.to(device)
        test_pred_logits = model(images)
        pred_labels_and_probs = {dic[i]: float(test_pred_logits[0][i]) for i in range(len(dic))}
        test_pred_labels = test_pred_logits.argmax(dim=1)
        print(dic[test_pred_logits.cpu().data.numpy().argmax()])

Broccoli
Cabbage
